# Extract variant carriers and perform annotation using GP2 WGS data (Release 8)

## Set Up

In [ ]:
## Use the os package to interact with the environment
import os

## Bring in Pandas for Dataframe functionality
import pandas as pd

import subprocess

## Numpy for basics
import numpy as np

## Use pathlib for file path manipulation
import pathlib

## Use StringIO for working with file contents
from io import StringIO

## Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

## Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

## Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

## BigQuery for querying data
from google.cloud import bigquery

## Import Sys
import sys as sys

### Install plink

In [ ]:
%%capture
%%bash

# Install plink 1.9
mkdir -p ~/tools
cd ~/tools/
if test -e ~/tools/plink; then
    echo "Plink is already installed"
else
    echo "Plink is not installed"
    cd ~/tools/

    wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 

    unzip -o plink_linux_x86_64_20190304.zip
    mv plink plink1.9
fi

In [ ]:
%%bash

# chmod plink 1.9 to make sure you have permission to run the program
chmod u+x ~/tools/plink1.9

In [ ]:
%%capture
%%bash

# Install plink 2.0
cd ~/tools/
if test -e ~/tools/plink2; then
    echo "Plink2 is already installed"
else
    echo "Plink2 is not installed"
    cd ~/tools/

    wget http://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip

    unzip -o plink2_linux_x86_64_latest.zip
fi

In [ ]:
%%bash

# chmod plink 2 to make sure you have permission to run the program
chmod u+x ~/tools/plink2

### Install bcftools

In [ ]:
%%capture
%%bash 

#install bcftools
cd /home/jupyter/tools/

if test -e /home/jupyter/tools/bcftools; then
    echo "bcftools is already installed in /home/jupyter/tools/"
else
    echo -e "Downloading bcftools \n    -------"
    git clone --recurse-submodules https://github.com/samtools/htslib.git
    git clone https://github.com/samtools/bcftools.git
    cd bcftools
    make
    echo -e "\n bcftools downloaded and unzipped in /home/jupyter/tools \n "

fi

In [ ]:
%%bash

# chmod bcftools to make sure you have permission to run the program
chmod +x /home/jupyter/tools/bcftools

### Alternative way to install bcftools (optional)

In [ ]:
# install bcftools through mamba

! mamba install -y -c bioconda bcftools

### Install annovar

In [ ]:
%%capture
%%bash

# Install ANNOVAR: We are adding the download link after registration on the annovar website
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e ~/tools/annovar ; then
    echo "annovar is already installed in /home/jupyter/tools/annovar"
else
    echo "annovar is not installed"
    cd ~/tools

    wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

    tar xvfz annovar.latest.tar.gz

fi

In [ ]:
%%bash

# Install ANNOVAR: Download resources for annotation

cd ~/tools/annovar
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20240917 humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp47a humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad41_genome humandb/


### Create working directory and set paths

In [ ]:
# Create a folder on your workspace
print("Making a working directory")
!mkdir -p /home/jupyter/workspace/ws_files/GP2_R8_dystonia
!mkdir -p /home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files
!mkdir -p /home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files
!mkdir -p /home/jupyter/workspace/ws_files/GP2_R8_dystonia/results

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

### Create bed file

#### Create a .txt file with the gene information (include all genes in which you want to extract variant carriers)
Use the following format: CHR START END GENE

CHR refers to the chromosome the gene is located on (e.g., chr1); START refers to the chromosomal position at which the gene starts; END refers to the chromosomal position at which the gene ends; GENE refers to the gene name (optional)

Use the ensembl genome browser to obtain these information (https://useast.ensembl.org/index.html)

In [ ]:
# Reformat the .txt file into a .bed file
input_file = "/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files/chrom_pos.txt"
output_file = "/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files/chrom_pos.bed"

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        cleaned_line = "\t".join(line.strip().split())  # Replace spaces with actual tabs
        outfile.write(cleaned_line + "\n")

print("Conversion complete! File saved as:", output_file)


In [ ]:
# Read in the bed file
bed = pd.read_csv('/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files/chrom_pos.bed',sep='\t',header=None,names=['chr','start_bp','stop_bp','gene'])
bed

In [ ]:
# fix chrX coding (Optional)
bed['chr']=bed['chr'].str.replace('chrx','chrX')

In [ ]:
# Write out bed for each chr, so we just need to run the plink for the chrom needed 
# instead of every gene

# create the directory where I store the bed files
dir_bed = '/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files'
os.makedirs(f'{dir_bed}', exist_ok=True)

for chr, dat in bed.groupby('chr'):
    dat=dat.sort_values('start_bp')
    dat.to_csv(f'{dir_bed}/{chr}.bed',sep='\t',header=False,index=False)

## Extract regions of interest per ancestry and generate merged files with all variant carriers

### AAC

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/path/to/release8/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AAC"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AAC"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AAC"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_AAC.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_AAC.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_AAC.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_AAC.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
AAC_df=anno[all_cols_to_keep]

#rename col
AAC_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

AAC_df

In [ ]:
# Count occurrences of each value
value_counts = AAC_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_AAC_df = AAC_df[AAC_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_AAC_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_AAC_df = filtered_AAC_df[filtered_AAC_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_AAC_df.head())

In [ ]:
# Save the filtered output
filtered_AAC_df.to_csv(f"{workdir}/results/filtered_multianno_AAC.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_AAC_df['var_id'].to_csv(f"{workdir}/results/AAC_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AAC"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno             

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AAC"

aac_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
aac_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=aac_var.columns[6:len(aac_var)]
d = aac_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=aac_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_AAC_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_AAC_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_AAC = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_AAC.to_csv(f"{workdir}/results/merged_genotypes_dystonia_AAC.tsv",sep='\t',index=False)

### AFR

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/path/to/release8/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AFR"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AFR"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AFR"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_AFR.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_AFR.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_AFR.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_AFR.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'gnomad41_genome_AF_afr',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
AFR_df=anno[all_cols_to_keep]

#rename col
AFR_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

AFR_df

In [ ]:
# Count occurrences of each value
value_counts = AFR_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_AFR_df = AFR_df[AFR_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_AFR_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_AFR_df = filtered_AFR_df[filtered_AFR_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_AFR_df.head())

In [ ]:
# Save the filtered output
filtered_AFR_df.to_csv(f"{workdir}/results/filtered_multianno_AFR.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_AFR_df['var_id'].to_csv(f"{workdir}/results/AFR_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AFR"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno              

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AFR"

afr_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
afr_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=afr_var.columns[6:len(afr_var)]
d = afr_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=afr_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_AFR_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_AFR_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_AFR = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_AFR.to_csv(f"{workdir}/results/merged_genotypes_dystonia_AFR.tsv",sep='\t',index=False)

### AJ

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/path/to/release8/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AJ"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AJ"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AJ"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_AJ.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_AJ.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_AJ.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_AJ.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'gnomad41_genome_AF_asj',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
AJ_df=anno[all_cols_to_keep]

#rename col
AJ_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

AJ_df

In [ ]:
# Count occurrences of each value
value_counts = AJ_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_AJ_df = AJ_df[AJ_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_AJ_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_AJ_df = filtered_AJ_df[filtered_AJ_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_AJ_df.head())

In [ ]:
# Save the filtered output
filtered_AJ_df.to_csv(f"{workdir}/results/filtered_multianno_AJ.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_AJ_df['var_id'].to_csv(f"{workdir}/results/AJ_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AJ"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno              

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AJ"

aj_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
aj_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=aj_var.columns[6:len(aj_var)]
d = aj_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=aj_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_AJ_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_AJ_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_AJ = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_AJ.to_csv(f"{workdir}/results/merged_genotypes_dystonia__AJ.tsv",sep='\t',index=False)

### AMR

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/path/to/release8/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AMR"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AMR"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AMR"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_AMR.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_AMR.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_AMR.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_AMR.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'gnomad41_genome_AF_amr',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
AMR_df=anno[all_cols_to_keep]

#rename col
AMR_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

AMR_df

In [ ]:
# Count occurrences of each value
value_counts = AMR_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_AMR_df = AMR_df[AMR_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_AMR_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_AMR_df = filtered_AMR_df[filtered_AMR_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_AMR_df.head())

In [ ]:
# Save the filtered output
filtered_AMR_df.to_csv(f"{workdir}/results/filtered_multianno_AMR.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_AMR_df['var_id'].to_csv(f"{workdir}/results/AMR_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AMR"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno             

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="AMR"

amr_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
amr_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=amr_var.columns[6:len(amr_var)]
d = amr_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=amr_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_AMR_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_AMR_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_AMR = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_AMR.to_csv(f"{workdir}/results/merged_genotypes_dystonia_AMR.tsv",sep='\t',index=False)

### CAH

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/path/to/release8/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAH"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAH"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAH"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_CAH.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_CAH.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_CAH.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_CAH.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
CAH_df=anno[all_cols_to_keep]

#rename col
CAH_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

CAH_df

In [ ]:
# Count occurrences of each value
value_counts = CAH_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_CAH_df = CAH_df[CAH_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_CAH_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_CAH_df = filtered_CAH_df[filtered_CAH_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_CAH_df.head())

In [ ]:
# Save the filtered output
filtered_CAH_df.to_csv(f"{workdir}/results/filtered_multianno_CAH.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_CAH_df['var_id'].to_csv(f"{workdir}/results/CAH_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAH"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno         

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAH"

cah_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
cah_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=cah_var.columns[6:len(cah_var)]
d = cah_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=cah_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_CAH_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_CAH_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_CAH = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_CAH.to_csv(f"{workdir}/results/merged_genotypes_dystonia_CAH.tsv",sep='\t',index=False)

### CAS

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/path/to/release8/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAS"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAS"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAS"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_CAS.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_CAS.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_CAS.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_CAS.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
CAS_df=anno[all_cols_to_keep]

#rename col
CAS_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

CAS_df

In [ ]:
# Count occurrences of each value
value_counts = CAS_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_CAS_df = CAS_df[CAS_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_CAS_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_CAS_df = filtered_CAS_df[filtered_CAS_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_CAS_df.head())

In [ ]:
# Save the filtered output
filtered_CAS_df.to_csv(f"{workdir}/results/filtered_multianno_CAS.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_CAS_df['var_id'].to_csv(f"{workdir}/results/CAS_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAS"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno              

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="CAS"

cas_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
cas_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=cas_var.columns[6:len(cas_var)]
d = cas_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=cas_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_CAS_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_CAS_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_CAS = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_CAS.to_csv(f"{workdir}/results/merged_genotype_dystonias_CAS.tsv",sep='\t',index=False)

### EAS

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/gp2_tier2_eu_release8_13092024/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EAS"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EAS"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EAS"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_EAS.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_EAS.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_EAS.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_EAS.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'gnomad41_genome_AF_eas',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
EAS_df=anno[all_cols_to_keep]

#rename col
EAS_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

EAS_df

In [ ]:
# Count occurrences of each value
value_counts = EAS_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_EAS_df = EAS_df[EAS_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_EAS_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_EAS_df = filtered_EAS_df[filtered_EAS_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_EAS_df.head())

In [ ]:
# Save the filtered output
filtered_EAS_df.to_csv(f"{workdir}/results/filtered_multianno_EAS.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_EAS_df['var_id'].to_csv(f"{workdir}/results/EAS_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EAS"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno            

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EAS"

eas_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
eas_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=eas_var.columns[6:len(eas_var)]
d = eas_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=eas_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_EAS_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_EAS_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'


In [ ]:
# get everything and write out
merged_df_EAS = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_EAS.to_csv(f"{workdir}/results/merged_genotypes_dystonia_EAS.tsv",sep='\t',index=False)

### EUR

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/gp2_tier2_eu_release8_13092024/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EUR"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EUR"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EUR"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_EUR.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_EUR.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_EUR.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_EUR.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'gnomad41_genome_AF_nfe',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
EUR_df=anno[all_cols_to_keep]

#rename col
EUR_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

EUR_df

In [ ]:
# Count occurrences of each value
value_counts = EUR_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_EUR_df = EUR_df[EUR_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_EUR_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_EUR_df = filtered_EUR_df[filtered_EUR_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_EUR_df.head())

In [ ]:
# Save the filtered output
filtered_EUR_df.to_csv(f"{workdir}/results/filtered_multianno_EUR.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_EUR_df['var_id'].to_csv(f"{workdir}/results/EUR_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EUR"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno              

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="EUR"

eur_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
eur_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=eur_var.columns[6:len(eur_var)]
d = eur_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=eur_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_EUR_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_EUR_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_EUR = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_EUR.to_csv(f"{workdir}/results/merged_genotypes_dystonia_EUR.tsv",sep='\t',index=False)

### FIN

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/gp2_tier2_eu_release8_13092024/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="FIN"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="FIN"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="FIN"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_FIN.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_FIN.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_FIN.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_FIN.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'gnomad41_genome_AF_fin',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
FIN_df=anno[all_cols_to_keep]

#rename col
FIN_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

FIN_df

In [ ]:
# Count occurrences of each value
value_counts = FIN_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_FIN_df = FIN_df[FIN_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_FIN_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_FIN_df = filtered_FIN_df[filtered_FIN_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_FIN_df.head())

In [ ]:
# Save the filtered output
filtered_FIN_df.to_csv(f"{workdir}/results/filtered_multianno_FIN.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_FIN_df['var_id'].to_csv(f"{workdir}/results/FIN_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="FIN"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno              

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="FIN"

fin_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
fin_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=fin_var.columns[6:len(fin_var)]
d = fin_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=fin_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_FIN_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_FIN_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)


In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'


In [ ]:
# get everything and write out
merged_df_FIN = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_FIN.to_csv(f"{workdir}/results/merged_genotypes_dystonia_FIN.tsv",sep='\t',index=False)

### MDE

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/gp2_tier2_eu_release8_13092024/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="MDE"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="MDE"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="MDE"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_MDE.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_MDE.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_MDE.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_MDE.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'gnomad41_genome_AF_mid',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
MDE_df=anno[all_cols_to_keep]

#rename col
MDE_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

MDE_df

In [ ]:
# Count occurrences of each value
value_counts = MDE_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_MDE_df = MDE_df[MDE_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_MDE_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_MDE_df = filtered_MDE_df[filtered_MDE_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_MDE_df.head())

In [ ]:
# Save the filtered output
filtered_MDE_df.to_csv(f"{workdir}/results/filtered_multianno_MDE.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_MDE_df['var_id'].to_csv(f"{workdir}/results/MDE_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="MDE"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno            

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="MDE"

mde_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
mde_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=mde_var.columns[6:len(mde_var)]
d = mde_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=mde_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_MDE_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_MDE_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_MDE = pd.concat([out_het,out_hom,comphet],axis=0)

# Save the dataset
merged_df_MDE.to_csv(f"{workdir}/results/merged_genotypes_dystonia_MDE.tsv",sep='\t',index=False)

### SAS

In [ ]:
# find unique chromosomes in bed files
unique_chr=bed['chr'].unique().tolist()
unique_chr

In [ ]:
#define all the variables

INPUT_PLINK_DIR="/home/jupyter/workspace/gp2_tier2_eu_release8_13092024/wgs/deepvariant_joint_calling/plink"
PLINK2_PATH="/home/jupyter/tools/plink2"
dir_bed="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/bed_files"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="SAS"

# extract variants we need from each chromosome
for chrom in unique_chr:    
    # make the output directory if not exist
    os.makedirs(f'{TEMP_DIR}/{Ancestry}', exist_ok=True)
    
    # Construct the command as a list 
    # keep only variants with maf <0.05 in the dataset to keep the files small
    # since frequent variants are not we are interested in
    cmd = [
        PLINK2_PATH,
        "--pfile", f"{INPUT_PLINK_DIR}/{Ancestry}/{chrom}_{Ancestry}_release8",
        "--mac", "1",
        "--extract", "bed1", f"{dir_bed}/{chrom}.bed",
        "--make-pgen",
        "--out", f"{TEMP_DIR}/{Ancestry}/{chrom}"
    ]

    subprocess.run(cmd, check=True)

In [ ]:
%%bash

TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="SAS"

# merge all the chrs and convert the final file to vcf
# for merging across pfiles https://www.cog-genomics.org/plink/2.0/data

# list the files, sort by chromosome and remove .pgen from the filename
ls ${TEMP_DIR}/${Ancestry}/*.pgen | sort -V  | sed 's/\.pgen//g' > ${TEMP_DIR}/${Ancestry}_pfiles.list

~/tools/plink2 --pmerge-list ${TEMP_DIR}/${Ancestry}_pfiles.list \
                           --recode vcf \
                           --out ${TEMP_DIR}/${Ancestry}

In [ ]:
%%bash
# bgzip the vcf
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="SAS"

bgzip -c ${TEMP_DIR}/${Ancestry}.vcf > ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz
tabix -p vcf ${TEMP_DIR}/annovar_input_${Ancestry}.vcf.gz

In [ ]:
%%bash

workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
mkdir -p ${workdir}/results

# Run ANNOVAR annotation
perl ~/tools/annovar/table_annovar.pl \
     ${workdir}/temp_files/annovar_input_SAS.vcf.gz \
    /home/jupyter/tools/annovar/humandb/ \
    -buildver hg38 \
    -out ${workdir}/results/wgs_final_SAS.annovar \
    -remove \
    -protocol refGene,gnomad41_genome,clinvar_20240917,dbnsfp47a \
    -operation g,f,f,f \
    -nastring . \
    -polish \
    -vcfinput

In [ ]:
%%bash 
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"

head ${workdir}/results/wgs_final_SAS.annovar.hg38_multianno.txt

In [ ]:
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
# Show columns of multianno.txt output file
anno=pd.read_csv(f'{workdir}/results/wgs_final_SAS.annovar.hg38_multianno.txt',sep='\t',dtype=str)

#find variant id col:
anno['Otherinfo6']

In [ ]:
anno.columns.tolist()

In [ ]:
# select the cols to keep
basic_cols= anno.columns.tolist()[0:10]
additional_cols_to_keep=['Otherinfo6',
                         'gnomad41_genome_fafmax_faf95_max',
                         'gnomad41_genome_AF_sas',
                         'CLNDN',
                         'CLNSIG',
                         'CADD_phred']
            
all_cols_to_keep= basic_cols+ additional_cols_to_keep
all_cols_to_keep

In [ ]:
# subset the columns
SAS_df=anno[all_cols_to_keep]

#rename col
SAS_df.rename({'Otherinfo6':'var_id'},axis=1,inplace=True)

SAS_df

In [ ]:
# Count occurrences of each value
value_counts = SAS_df["ExonicFunc.refGene"].value_counts()

# Display counts
print(value_counts)

#### Filter variants

In [ ]:
# Define the values you want to keep
keep_values = ["exonic", "splicing", "exonic;splicing"]  # Add more if needed

# Subset the dataframe
filtered_SAS_df = SAS_df[SAS_df["Func.refGene"].isin(keep_values)]

# Display the filtered dataframe
print (filtered_SAS_df.head())

In [ ]:
# Filter out synonymous SNVs
filtered_SAS_df = filtered_SAS_df[filtered_SAS_df["ExonicFunc.refGene"] != "synonymous SNV"]

# Display the filtered dataframe
print (filtered_SAS_df.head())

In [ ]:
# Save the filtered output
filtered_SAS_df.to_csv(f"{workdir}/results/filtered_multianno_SAS.tsv", sep="\t", index=False)

# write out var_id to extract from plink files
filtered_SAS_df['var_id'].to_csv(f"{workdir}/results/SAS_var_to_extract.txt",index=False,header=False)

#### Extract carrier IDs and genotypes

I would just extract the variants you are interested from the annovar annotation, and you can start from the plink files generated from the merge

In [ ]:
%%bash
workdir="/home/jupyter/workspace/ws_files/GP2_R8_dystonia"
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_dystonia/temp_files"
Ancestry="SAS"

~/tools/plink2 --pfile ${TEMP_DIR}/${Ancestry} \
               --extract ${workdir}/results/${Ancestry}_var_to_extract.txt \
               --recode A \
               --out ${TEMP_DIR}/${Ancestry}_geno              

In [ ]:
TEMP_DIR="/home/jupyter/workspace/ws_files/GP2_R8_monogenic/temp_files"
Ancestry="SAS"

sas_var = pd.read_csv(f'{TEMP_DIR}/{Ancestry}_geno.raw', sep='\s+')
sas_var

From here you can run similar code for NBA to collect variant carriers 

In [ ]:
# transpose the dataframe to be row as variants and columns as samples
var_col=sas_var.columns[6:len(sas_var)]
d = sas_var.drop(columns=['FID','PAT','MAT','SEX','PHENOTYPE'])
sample=sas_var[['IID','PHENOTYPE']]

#Filtering rows where any value in var_col is ≤1 (either het or hom)
t=d[(d[var_col]<=1).any(axis=1)].T
t.columns = t.iloc[0]
t=t.iloc[1:]
t.reset_index(inplace=True)

t

In [ ]:
#strip the last '_${ref_allele}', so we can keep the same variant id as in annotation 
t['index'] = t['index'].str.rsplit('_', n=1).str[0]
t.rename({'index':'var_id'},axis=1,inplace=True)
t

In [ ]:
t['hom_carrier'] = t.apply(lambda row: row[row == 0].index.tolist() , axis=1)
t['het_carrier'] = t.apply(lambda row: row[row == 1].index.tolist() , axis=1)  

# store hom and het seperately to later explode the dataframe correctly
hom = t[['var_id','hom_carrier']]
het = t[['var_id','het_carrier']]

In [ ]:
# check hom as example what to expect
hom

In [ ]:
# split the carrier id from the list and each to a row
hom = hom.explode('hom_carrier', ignore_index=True)

hom

In [ ]:
# keep just the variants with carriers
hom= hom.loc[~hom['hom_carrier'].isnull()]

hom

In [ ]:
#merge with annotation
out_hom = pd.merge(hom,filtered_SAS_df, on='var_id',how='left')
out_hom['zygosity'] = 'hom'

#rename col
out_hom.rename({'hom_carrier':'carrier_id'},axis=1,inplace=True)

out_hom

In [ ]:
#repeat the same for het
het = het.explode('het_carrier', ignore_index=True)
het= het.loc[~het['het_carrier'].isnull()]

het

In [ ]:
#merge with annotation
out_het = pd.merge(het,filtered_SAS_df, on='var_id',how='left')
out_het['zygosity'] = 'het'

#rename col
out_het.rename({'het_carrier':'carrier_id'},axis=1,inplace=True)

out_het

In [ ]:
# check if there's any comphet by grouping gene and sample_id
pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)

In [ ]:
# write out comphet
comphet = pd.concat(g for _, g in out_het.groupby(["Gene.refGene","carrier_id"]) if len(g) > 1)
comphet['zygosity'] = 'comphet'

In [ ]:
# get everything and write out
merged_df_SAS = pd.concat([out_het,out_hom],axis=0)

# Save the dataset
merged_df_SAS.to_csv(f"{workdir}/results/merged_genotypes_dystonia_SAS.tsv",sep='\t',index=False)